<h1 align="center">Computación de Alto Desempeño</h1>
<h1 align="center">OpenMP - Problemas de Rendimiento</h1>
<h1 align="center">2024</h1>
<h1 align="center">MEDELLÍN - COLOMBIA </h1>

***
|[![Outlook](https://img.shields.io/badge/Microsoft_Outlook-0078D4?style=plastic&logo=microsoft-outlook&logoColor=white)](mailto:calvarezh@udemedellin.edu.co)||[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/carlosalvarezh/HPC/blob/main/HPC11_OpenMP_PerformanceIssues.ipynb)
|-:|:-|--:|
|[![LinkedIn](https://img.shields.io/badge/linkedin-%230077B5.svg?style=plastic&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/carlosalvarez5/)|[![@alvarezhenao](https://img.shields.io/twitter/url/https/twitter.com/alvarezhenao.svg?style=social&label=Follow%20%40alvarezhenao)](https://twitter.com/alvarezhenao)|[![@carlosalvarezh](https://img.shields.io/badge/github-%23121011.svg?style=plastic&logo=github&logoColor=white)](https://github.com/carlosalvarezh)|

<table>
 <tr align=left><td><img align=left src="https://github.com/carlosalvarezh/Curso_CEC_EAFIT/blob/main/images/CCLogoColorPop1.gif?raw=true" width="25">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license.(c) Carlos Alberto Alvarez Henao</td>
</table>

***

## **Problemas de rendimiento en OpenMP**

Cuando trabajamos con OpenMP, es importante no solo paralelizar el código, sino también garantizar que el rendimiento sea óptimo. Aunque OpenMP facilita la paralelización, existen varios factores que pueden afectar el rendimiento del código, como la topología de los procesadores, la afinidad de los *threads*, la sincronización, la serialización y los desequilibrios de carga. A continuación, se abordan estos problemas y cómo enfrentarlos.

### **Problemas de topología y afinidad**

En sistemas modernos, los procesadores pueden tener múltiples núcleos, y cada núcleo puede tener su propia jerarquía de memoria. **La topología de los procesadores** se refiere a cómo están organizados estos núcleos dentro de la CPU, y entenderla es crucial para optimizar el rendimiento. La forma en que los *threads* se asignan a los núcleos puede afectar significativamente el rendimiento, ya que una distribución ineficiente puede provocar problemas con la caché y la transferencia de datos entre los núcleos.

**Afinidad de los *threads***

La **afinidad de los *threads*** se refiere a la relación entre los *threads* de OpenMP y los núcleos físicos del procesador. Si los *threads* no están asignados correctamente a los núcleos, es posible que se produzcan penalizaciones en el rendimiento debido a la migración de *threads* entre núcleos. Esto sucede cuando los *threads* acceden a datos que residen en la caché de otro núcleo, lo que genera tiempos de espera adicionales.

**Ejemplo de Problema de Afinidad**

En un sistema con múltiples procesadores, cada procesador tiene su propia memoria caché. Si los *threads* están mal distribuidos y acceden frecuentemente a datos almacenados en la caché de otro procesador, habrá un gran tiempo de espera para transferir esos datos, afectando negativamente el rendimiento general.

**Solución: Ajustar la Afinidad**

OpenMP proporciona variables de entorno para controlar la afinidad de los *threads*. La variable `OMP_PROC_BIND` asegura que los *threads* permanezcan en núcleos específicos, evitando la migración entre ellos. Esto optimiza el uso de la caché.

```bash
export OMP_PROC_BIND=TRUE  # Asegura que los threads permanezcan atados a sus núcleos asignados
```

Otra variable útil es `OMP_PLACES`, que permite especificar los núcleos donde se ejecutarán los *threads*.

```bash
export OMP_PLACES=cores  # Asigna un thread por cada núcleo físico
```

**Ejemplo: Optimización de la Afinidad en un Bucle Paralelo**

```c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define N 100000000

int main() {
    double *a = (double *)malloc(N * sizeof(double));
    double *b = (double *)malloc(N * sizeof(double));
    double *c = (double *)malloc(N * sizeof(double));

    // Inicializar los vectores con valores
    for (int i = 0; i < N; i++) {
        a[i] = i * 0.5;
        b[i] = i * 1.5;
    }

    // Configurar la afinidad de los threads
    omp_set_num_threads(16);  // 4 procesadores con 4 núcleos cada uno
    omp_set_nested(1);  // Permitir anidamiento de threads

    // Paralelizar el cálculo con OpenMP
    double start_time = omp_get_wtime();  // Medir tiempo de ejecución
    #pragma omp parallel
    {
        #pragma omp for schedule(static)
        for (int i = 0; i < N; i++) {
            c[i] = a[i] + b[i];  // Sumar los arreglos
        }
    }
    double end_time = omp_get_wtime();

    printf("Tiempo de ejecución: %.6f segundos\n", end_time - start_time);

    // Liberar memoria
    free(a);
    free(b);
    free(c);

    return 0;
}
```

**Explicación del Ejemplo**

1. **Asignación de Afinidad de *threads***: Se usa `omp_set_num_threads(16)` para configurar el número de *threads* y `omp_set_nested(1)` para permitir la asignación eficiente a los núcleos.
2. **Medición de Tiempo**: Con `omp_get_wtime()`, se mide el tiempo de ejecución, lo que permite ver cómo la afinidad afecta el rendimiento del bucle paralelo.
3. **Optimización con Variables de Entorno**: Ejecutando el programa con las variables de entorno `OMP_PROC_BIND=TRUE` y `OMP_PLACES=cores`, se asegura que los *threads* estén correctamente asignados, optimizando el rendimiento al evitar la migración entre núcleos.

### **Sincronización y serialización**

En la programación paralela, la **sincronización** y la **serialización** son dos aspectos clave que pueden afectar el rendimiento si no se gestionan adecuadamente. Aunque son necesarias para asegurar la correcta ejecución de tareas concurrentes, su mal uso puede introducir problemas de eficiencia.

**Problema de Sincronización**

La **sincronización** es fundamental para evitar condiciones de carrera, que ocurren cuando múltiples *threads* intentan acceder o modificar simultáneamente un recurso compartido. Sin embargo, cada vez que los *threads* deben sincronizarse (por ejemplo, en una sección crítica o en una barrera), algunos de ellos se ven obligados a esperar, lo que genera tiempos muertos y retrasa la ejecución del programa. Este tiempo de espera puede reducir significativamente los beneficios del paralelismo.

**Problema de Serialización**

La **serialización** ocurre cuando un fragmento de código que debería ejecutarse en paralelo se ve obligado a ejecutarse de forma secuencial. Esto sucede con frecuencia cuando se utiliza una sincronización excesiva o ineficiente, lo que limita la capacidad de los *threads* de trabajar en paralelo. La serialización también puede aparecer cuando ciertas secciones del código no pueden paralelizarse adecuadamente y los *threads* deben esperar por el acceso exclusivo a recursos compartidos.

**Solución: Minimizar la Sincronización y Evitar la Serialización**

1. **Usar sincronización solo cuando sea absolutamente necesaria**: Directivas como `#pragma omp critical` y `#pragma omp atomic` son útiles para proteger el acceso a variables compartidas, pero su uso excesivo puede afectar gravemente el rendimiento. Solo deberías sincronizar cuando sea estrictamente necesario para evitar condiciones de carrera.

2. **Utilizar la directiva `reduction` cuando sea posible**: Para operaciones como sumas o acumulaciones, `reduction` es una solución mucho más eficiente que `critical` o `atomic`, ya que permite que cada *thread* realice una parte de la operación en paralelo y solo combine los resultados al final, evitando la serialización.

   **Ejemplo de uso de `reduction`:**
   ```c
   #pragma omp parallel for reduction(+:total)
   for (int i = 0; i < N; i++) {
       total += array[i];  // Paraleliza la suma de los elementos de un arreglo
   }
   ```

3. **Evitar serialización innecesaria**: Es fundamental revisar el código paralelo para eliminar tareas que se ejecuten de manera secuencial dentro de bloques paralelos. Por ejemplo, reducir el uso de `#pragma omp single` cuando no sea necesario, ayuda a evitar cuellos de botella y mejora el paralelismo efectivo.

**Ejemplo Práctico: Reducción de la Serialización Usando `reduction`**

```c
#include <stdio.h>
#include <omp.h>
#define N 1000000

int main() {
    double a[N], total = 0.0;

    // Inicializar el arreglo
    for (int i = 0; i < N; i++) {
        a[i] = i * 0.5;
    }

    // Paralelizar con reducción en lugar de critical
    double start_time = omp_get_wtime();

    #pragma omp parallel for reduction(+:total)
    for (int i = 0; i < N; i++) {
        total += a[i];  // Sumar los elementos del arreglo
    }

    double end_time = omp_get_wtime();

    printf("Suma total: %.2f\n", total);
    printf("Tiempo de ejecución (con reduction): %.6f segundos\n", end_time - start_time);

    return 0;
}
```

**Explicación del Ejemplo**

1. **Uso de `reduction`**: En este código, `reduction(+:total)` permite que cada *thread* realice sumas parciales de los elementos del arreglo de manera independiente. Solo al final, los resultados se combinan, eliminando la necesidad de usar `#pragma omp critical`, lo que mejora el paralelismo.

2. **Medición del tiempo**: El uso de `reduction` minimiza la serialización y, por lo tanto, mejora el rendimiento general del programa. Esto se puede observar al medir el tiempo de ejecución con `omp_get_wtime()`, comparando el rendimiento con y sin serialización.

### **Resumen**
La clave para optimizar el rendimiento en la programación paralela es **minimizar la sincronización innecesaria** y **evitar la serialización**. Usar herramientas como `reduction` para operaciones comunes como la suma y evitar tareas secuenciales dentro de bloques paralelos son prácticas que contribuyen a mantener un alto grado de paralelismo y maximizar el rendimiento de las aplicaciones concurrentes.

### **Otros sobrecostos de OpenMP**

Además de los problemas de sincronización y afinidad, existen otros **sobrecostos asociados con OpenMP** que pueden afectar el rendimiento general de los programas paralelos. Estos incluyen la creación y destrucción de *threads*, la programación especulativa, y los problemas relacionados con el uso de la caché.

**Sobrecosto en la Creación de *threads***

Uno de los principales desafíos en OpenMP es el **sobrecosto que implica la creación y destrucción de bloques paralelos**. Cada vez que se ejecuta un bloque `#pragma omp parallel`, OpenMP tiene que iniciar *threads*, lo que puede impactar negativamente en el rendimiento si esto se hace con demasiada frecuencia.

**Solución: Agrupar Tareas en un Único Bloque Paralelo**

Para minimizar el **sobrecosto de creación y destrucción de *threads***, es recomendable agrupar varias tareas dentro de un mismo bloque paralelo. Al reutilizar los *threads* en lugar de crear y destruir nuevos para cada operación, se evita el gasto innecesario de tiempo. En algunos casos, también puedes usar "bloques paralelos anidados" para realizar múltiples tareas sin necesidad de crear continuamente nuevos *threads*.

**Ejemplo de Agrupación de Tareas en un Solo Bloque Paralelo**:
```c
#pragma omp parallel
{
    #pragma omp for
    for (int i = 0; i < N; i++) {
        // Trabajo en paralelo
    }

    #pragma omp for
    for (int j = 0; j < M; j++) {
        // Otro trabajo en paralelo, mismo bloque de threads
    }
}
```

En este ejemplo, todas las tareas se realizan dentro del mismo bloque paralelo, reutilizando los *threads* y evitando el coste de su creación y destrucción para cada tarea.

**Sobrecosto de la Programación Especulativa**

Otro sobrecosto en OpenMP puede surgir debido a la **programación especulativa**, donde el sistema intenta ejecutar tareas antes de que estén completamente listas. Esto genera ineficiencia, ya que si las dependencias aún no están resueltas, el sistema debe recalcular o detener la ejecución hasta que las dependencias estén disponibles.

**Sobrecosto de *Cache Misses***

Cuando varios *threads* acceden a la misma caché de datos, pueden producirse **cache misses**. Esto ocurre cuando los datos requeridos por un *thread* no están en la caché local y deben transferirse desde la memoria principal, lo que ralentiza el rendimiento.

**Solución: Localidad de Datos**

Para mitigar los **cache misses**, es importante optimizar la **localidad de datos**. Asegurar que los *threads* trabajen con datos que estén cercanos en la memoria y alineados correctamente mejora el rendimiento. Utilizar la variable de entorno `OMP_PLACES` permite asociar *threads* a núcleos que compartan caché, lo que puede reducir los tiempos de acceso a la memoria.

**Ejemplo: Agrupación de Tareas en un Bloque Paralelo**

```c
#include <stdio.h>
#include <omp.h>
#define N 100000

int main() {
    double a[N], b[N], c[N], total = 0.0;

    // Inicializar los arreglos
    for (int i = 0; i < N; i++) {
        a[i] = i * 0.5;
        b[i] = i * 1.5;
    }

    // Crear un solo bloque paralelo grande
    double start_time = omp_get_wtime();
    
    #pragma omp parallel
    {
        // Primera tarea: Sumar los elementos de dos arreglos
        #pragma omp for
        for (int i = 0; i < N; i++) {
            c[i] = a[i] + b[i];
        }

        // Segunda tarea: Calcular la suma total de un arreglo
        #pragma omp for reduction(+:total)
        for (int i = 0; i < N; i++) {
            total += c[i];
        }
    }

    double end_time = omp_get_wtime();

    printf("Suma total: %.2f\n", total);
    printf("Tiempo de ejecución: %.6f segundos\n", end_time - start_time);

    return 0;
}
```

**Explicación del Ejemplo**

1. **Reutilización de *threads***: El código utiliza un solo bloque `#pragma omp parallel` para realizar dos tareas diferentes. Los *threads* se crean una vez y se reutilizan, lo que minimiza el sobrecosto asociado con la creación y destrucción repetida de *threads*.
   
2. **Medición del tiempo**: Al medir el tiempo de ejecución con `omp_get_wtime()`, se puede observar que agrupar las tareas en un solo bloque paralelo mejora el rendimiento general, especialmente en programas que requieren múltiples cálculos paralelos.

**Conclusión**

Para optimizar el rendimiento en OpenMP, es esencial reducir los **sobrecostos asociados con la creación de *threads***, la **programación especulativa**, y los **cache misses**. Al agrupar tareas en bloques paralelos y mejorar la **localidad de datos**, puedes evitar muchos de estos sobrecostos y mejorar significativamente la eficiencia del paralelismo.

### **Luchando contra el desequilibrio de carga (Load Imbalance)**

**¿Qué es el desequilibrio de carga?**

El **desequilibrio de carga** ocurre cuando algunos *threads* están realizando más trabajo que otros. En un escenario ideal, el trabajo debería dividirse de manera equitativa entre todos los *threads* para maximizar el rendimiento. Sin embargo, cuando ciertos *threads* terminan sus tareas antes que otros, los *threads* inactivos deben esperar, lo que provoca ineficiencias y tiempos muertos, reduciendo el aprovechamiento total del paralelismo.

**Solución: Utilizar Programación Dinámica**

OpenMP ofrece diferentes estrategias para repartir el trabajo de los bucles. El esquema de programación predeterminado es **estático**, donde el trabajo se divide en partes iguales entre los *threads*. Aunque este método puede ser eficaz cuando las iteraciones tienen una carga de trabajo homogénea, no siempre es ideal. Si las iteraciones varían en complejidad, se recomienda el uso de **programación dinámica** o **guided**, que asignan las iteraciones de manera más eficiente.

- **Estática**: El trabajo se reparte equitativamente al inicio, lo que es eficiente cuando todas las iteraciones tienen tiempos de ejecución similares.
  
  ```c
  #pragma omp parallel for schedule(static)
  ```

- **Dinámica**: Asigna iteraciones de forma dinámica conforme los *threads* van completando su trabajo, lo que es útil cuando las iteraciones varían en tiempo de procesamiento.
  
  ```c
  #pragma omp parallel for schedule(dynamic)
  ```

- **Guided**: Al inicio, asigna grandes bloques de trabajo a los *threads* y va disminuyendo el tamaño de los bloques conforme avanza la ejecución. Es útil cuando la carga de trabajo es impredecible.

  ```c
  #pragma omp parallel for schedule(guided)
  ```

**Ejemplo de Desequilibrio de Carga**

Considera un bucle en el que las primeras iteraciones son más costosas que las últimas. Si se utiliza una programación estática, algunos *threads* podrían quedarse inactivos mientras los otros aún están trabajando. Aquí es donde `schedule(dynamic)` puede ser útil, ya que asigna nuevas iteraciones a los *threads* tan pronto como terminan su trabajo anterior, manteniéndolos activos.

```c
#pragma omp parallel for schedule(dynamic)
for (int i = 0; i < N; i++) {
    // Trabajo con tiempos de procesamiento variables
}
```

**Ejemplo Práctico: Uso de `schedule(dynamic)` para Equilibrar la Carga**

```c
#include <stdio.h>
#include <omp.h>
#include <unistd.h>  // Para la función sleep

#define N 1000

int main() {
    int a[N];

    // Inicializar el arreglo
    for (int i = 0; i < N; i++) {
        a[i] = i;  // Algunas iteraciones tendrán más trabajo
    }

    // Procesar el arreglo con balanceo de carga dinámico
    double start_time = omp_get_wtime();

    #pragma omp parallel for schedule(dynamic)
    for (int i = 0; i < N; i++) {
        // Simular diferentes tiempos de trabajo
        if (a[i] < N / 2) {
            sleep(1);  // Las primeras iteraciones toman más tiempo
        }
        printf("Thread %d procesó el elemento %d\n", omp_get_thread_num(), a[i]);
    }

    double end_time = omp_get_wtime();

    printf("Tiempo de ejecución: %.6f segundos\n", end_time - start_time);

    return 0;
}
```

**Explicación del Ejemplo**

1. **Simulación de tiempos desiguales**: En este bucle, las primeras iteraciones toman más tiempo de procesamiento que las últimas, simulando una carga de trabajo no homogénea.
   
2. **Uso de `schedule(dynamic)`**: Al emplear la programación dinámica, las iteraciones se asignan de manera continua a los *threads* conforme completan su trabajo. Los *threads* más rápidos toman nuevas iteraciones en lugar de quedarse inactivos esperando a que los más lentos terminen.

3. **Mejora del balance de carga**: La programación dinámica distribuye mejor las iteraciones y ayuda a equilibrar la carga entre los *threads*, reduciendo los tiempos de inactividad y mejorando el rendimiento global del programa.

**Conclusión**

El desequilibrio de carga es un desafío común en la programación paralela. Para combatirlo, OpenMP ofrece estrategias como la **programación dinámica** y **guided**, que permiten asignar el trabajo de manera más eficiente entre los *threads*. Ajustar la programación según la naturaleza del problema puede evitar que algunos *threads* queden inactivos, maximizando el rendimiento y la eficiencia del programa paralelo.

### **Resumen**



Los problemas de rendimiento en OpenMP, como la afinidad de los *threads*, la sincronización excesiva, los sobrecostos y el desequilibrio de carga, pueden afectar significativamente la eficiencia de los programas paralelos. Sin embargo, utilizando las estrategias correctas, como la optimización de la afinidad, el uso de `reduction`, la agrupación de tareas en bloques paralelos y la programación dinámica, podemos mejorar considerablemente el rendimiento. Los ejemplos prácticos presentados aquí ofrecen soluciones claras y aplicables a estos problemas comunes en OpenMP.

## **Ejercicios**

### **1. Impacto de la Afinidad de Threads en el Rendimiento**

1. Escribe un programa en C que realice una operación intensiva en CPU, como la multiplicación de matrices.
2. Ejecuta el programa con diferentes configuraciones de afinidad de *threads* utilizando las variables de entorno `OMP_PROC_BIND` y `OMP_PLACES`.
3. Mide el tiempo de ejecución en cada configuración y compáralos.

**Pasos a seguir**:
- Escribe un programa que multiplique dos matrices de gran tamaño.
- Usa las siguientes configuraciones de variables de entorno para ejecutar el programa:
  - **Configuración 1**: 
    ```bash
    export OMP_PROC_BIND=false
    export OMP_PLACES=threads
    ./nombre_del_programa
    ```
  - **Configuración 2**:
    ```bash
    export OMP_PROC_BIND=true
    export OMP_PLACES=cores
    ./nombre_del_programa
    ```
- Mide el tiempo de ejecución en cada caso con `omp_get_wtime()` y analiza cómo la afinidad afecta el rendimiento.


### **2. Comparación de Métodos de Sincronización**

1. Implementa un programa que sume los elementos de un arreglo grande.
2. Escribe dos versiones:
   - Una que use `#pragma omp critical` para proteger la variable compartida.
   - Otra que use `#pragma omp reduction` para sumar los valores de manera paralela.
3. Mide el tiempo de ejecución en cada versión y compáralos.

**Pasos a seguir**:
- Implementa una versión del código donde uses `#pragma omp critical`:
   ```c
   #pragma omp parallel for
   for (int i = 0; i < N; i++) {
       #pragma omp critical
       total += array[i];
   }
   ```
- Implementa una segunda versión con `#pragma omp reduction`:
   ```c
   #pragma omp parallel for reduction(+:total)
   for (int i = 0; i < N; i++) {
       total += array[i];
   }
   ```
- Ejecuta ambas versiones del programa y mide el tiempo de ejecución para cada una.
- Compara los resultados y discute cuál es más eficiente y por qué.

### **3. Sobrecosto por Creación de Bloques Paralelos**

1. Escribe un programa que realice varias tareas pequeñas en bloques `#pragma omp parallel` independientes.
2. Luego, optimiza el programa agrupando todas las tareas dentro de un solo bloque paralelo.
3. Mide el tiempo de ejecución antes y después de la optimización.

**Pasos a seguir**:
- Primero, escribe un programa que cree múltiples bloques paralelos:
   ```c
   for (int i = 0; i < TAREAS; i++) {
       #pragma omp parallel
       {
           // Cada bloque realiza una tarea pequeña
           realizarTareaPequena(i);
       }
   }
   ```
- Luego, optimiza el programa agrupando todas las tareas en un solo bloque:
   ```c
   #pragma omp parallel
   {
       #pragma omp for
       for (int i = 0; i < TAREAS; i++) {
           realizarTareaPequena(i);
       }
   }
   ```
- Mide el tiempo de ejecución antes y después de la optimización y explica cómo reducir la creación de bloques paralelos mejora el rendimiento.


### **4. Efecto de `schedule` en el Desequilibrio de Carga**

1. Implementa un bucle en el que el tiempo de procesamiento de las iteraciones varía.
2. Ejecuta el bucle con diferentes esquemas de programación: estática, dinámica, y guided.
3. Mide el tiempo de ejecución en cada caso y compáralos.

**Pasos a seguir**:
- Implementa un bucle en el que las primeras iteraciones tomen más tiempo de procesamiento que las últimas:
   ```c
   #pragma omp parallel for schedule(static)
   for (int i = 0; i < N; i++) {
       if (i < N/2) {
           sleep(1);  // Simula un trabajo más pesado en las primeras iteraciones
       }
       procesar(i);
   }
   ```
- Repite el mismo código con `schedule(dynamic)` y `schedule(guided)`:
   ```c
   #pragma omp parallel for schedule(dynamic)
   ```

   ```c
   #pragma omp parallel for schedule(guided)
   ```
- Mide el tiempo de ejecución en cada caso y explica cómo cambia el rendimiento según el esquema de programación utilizado.


### **5. Minimización de `cache misses` mediante Localidad de Datos**

**Objetivo**: Mejorar el acceso a la memoria y reducir los *cache misses* reestructurando el acceso a los datos.

**Descripción del Ejercicio**:
1. Escribe un programa que acceda a una matriz de manera no contigua (acceso a columnas).
2. Reestructura el programa para acceder a la matriz de manera contigua (acceso a filas).
3. Mide el tiempo de ejecución antes y después de la optimización y compáralos.

**Pasos a seguir**:
- Primero, implementa un código que acceda a los datos por columnas (acceso no contiguo):
   ```c
   #pragma omp parallel for
   for (int i = 0; i < N; i++) {
       for (int j = 0; j < M; j++) {
           matriz[j][i] += 1.0;  // Acceso columna por columna
       }
   }
   ```
- Luego, reestructura el código para acceder a los datos por filas (acceso contiguo):
   ```c
   #pragma omp parallel for
   for (int i = 0; i < N; i++) {
       for (int j = 0; j < M; j++) {
           matriz[i][j] += 1.0;  // Acceso fila por fila
       }
   }
   ```
- Mide el tiempo de ejecución en ambos casos y explica cómo el acceso contiguo mejora el rendimiento al reducir los *cache misses*.